<a href="https://colab.research.google.com/github/sayanbanerjee32/multimodal_llm/blob/main/phi_3_QLoRA_instruct150k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install transformers==4.44.2
!pip install -Uq accelerate peft bitsandbytes trl dataset transformers
# !pip install -Uq flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [ ]:
import numpy as np
import copy
import requests
from tqdm import tqdm
import os, gc
import subprocess
import json
import random
### Download Phi-3 model
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, PreTrainedModel
from transformers.trainer_callback import TrainerCallback
from torch.utils.data import DataLoader
from datasets import Dataset, DatasetDict
import joblib
from huggingface_hub import hf_hub_download

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from peft.tuners import lora

In [ ]:
# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### Download Phi-3 model

In [ ]:
# Load the Phi-3 model and tokenizer
model_name = "microsoft/Phi-3.5-mini-instruct"
# "microsoft/Phi-3-mini-4k-instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="right",
#                                            trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'left'

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

### Downlaod image embedding

In [ ]:
# URL of the embeddings file (replace with your actual URL)
embeddings_url = '/content/drive/MyDrive/multimodal_llm/image_embedding/coco_image_embeddings.npz'

# Load the embeddings
print("Loading embeddings...")
embeddings = np.load(embeddings_url, allow_pickle=True)

# Print embeddings and image names
for image_name, embedding in embeddings.items():
    print(f"Image: {image_name}")
    print(f"Embedding shape: {embedding.shape}")
    print(f"Embedding preview: {embedding[:5]}...")  # Print first 5 values
    print("-" * 50)
    break

print(f"Total number of embeddings: {len(embeddings)}")

Loading embeddings...
Image: 000000401144.jpg
Embedding shape: (512,)
Embedding preview: [-0.13     0.1564   0.02017  0.1678   0.2393 ]...
--------------------------------------------------
Total number of embeddings: 81479


### Data processing

In [ ]:
# List of URLs to download
url = "https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/resolve/main/llava_instruct_150k.json"

# Download each file
subprocess.run(["wget", "-c", url])

# Load the downloaded JSON file
json_file = "llava_instruct_150k.json"
with open(json_file, 'r') as f:
    data = json.load(f)

# Function to convert conversation format
# def convert_conversation(conversation):
#     system_message = "<|system|>\nYou are a helpful assistant.<|end|>\n"
#     user_message = ""
#     assistant_message = ""

#     for item in conversation:
#         if item['from'] == 'human':
#             user_message = f"<|user|>\n{item['value']}<|end|>\n"
#         elif item['from'] == 'gpt':
#             assistant_message = f"<|assistant|>\n{item['value']}<|end|>\n"

#     return system_message + user_message + assistant_message

# Process and tokenize the data
# Process and tokenize the data


# tokenized_data = []
# for item in tqdm(data, desc="Tokenizing data"):
#     image_file = item['image']
#     if image_file in embeddings:
#         image_embedding = torch.tensor(embeddings[image_file], dtype=torch.float32, device=device)

#         # Use the existing conversation format
#         conversation = [
#             {"role": "system", "content": "You are a helpful assistant."}
#         ] + [{"role": "user" if msg['from'] == 'human' else "assistant", "content": msg['value']}
#              for msg in item['conversations']]

#         # Apply chat template and tokenize directly
#         tokenized_conversation = tokenizer.apply_chat_template(conversation, return_tensors='pt').to(device)

#         tokenized_item = {
#             'image': image_file,
#             'image_embedding': image_embedding,
#             'tokenized_conversation': tokenized_conversation
#         }
#         tokenized_data.append(tokenized_item)

# print(f"Total tokenized items: {len(tokenized_data)}")
# print(f"Sample tokenized item:")
# print(f"Image: {tokenized_data[0]['image']}")
# print(f"Image embedding shape: {tokenized_data[0]['image_embedding'].shape}")
# print(f"Tokenized conversation shape: {tokenized_data[0]['tokenized_conversation'].shape}")
# print(f"Image embedding device: {tokenized_data[0]['image_embedding'].device}")
# print(f"Tokenized conversation device: {tokenized_data[0]['tokenized_conversation'].device}")

In [ ]:
data = data[0:10000]

In [ ]:
def create_dataset():
    processed_data = []
    print("Processing data...")
    with tqdm(total=len(data)) as pbar:
        for item in data:
            image_file = item['image']
            if image_file in embeddings:
                processed_data.append({
                    'image': image_file,
                    'image_embedding': embeddings[image_file].tolist(),
                    'conversation': item['conversations']
                })
            pbar.update(1)

    print(f"Data processing completed. Total processed items: {len(processed_data)}")

    return Dataset.from_dict({
        "image": [item['image'] for item in processed_data],
        "image_embedding": [item['image_embedding'] for item in processed_data],
        "conversation": [item['conversation'] for item in processed_data]
    })

print("Creating HuggingFace dataset...")
hf_dataset = create_dataset()

print("HuggingFace dataset creation completed.")
print(f"Total samples in dataset: {len(hf_dataset)}")



Creating HuggingFace dataset...
Processing data...


100%|██████████| 10000/10000 [01:27<00:00, 114.88it/s]


Data processing completed. Total processed items: 10000
HuggingFace dataset creation completed.
Total samples in dataset: 10000


In [ ]:
# print("Applying tokenization and preparing the dataset...")

# def prepare_dataset(examples):
#     image_embeddings = torch.stack([torch.tensor(item) for item in examples['image_embedding']])

#     conversations = []
#     for conv in examples['conversation']:
#         dialogue = [{"role": "system", "content": "You are a helpful assistant."}]

#         for i, message in enumerate(conv):
#             if message['from'] == 'human':
#                 content = message['value'].replace('<image>', '').strip()  # Remove '<image>' and strip whitespace
#                 if i == 0:
#                     content = f"Given the following information, provide a detailed and accurate response:\n{content}\n[An image is provided for this task.]\n"
#                 dialogue.append({"role": "user", "content": content})
#             elif message['from'] == 'gpt':
#                 dialogue.append({"role": "assistant", "content": message['value']})

#         conversations.append(dialogue)

#     tokenized_conversations = tokenizer.apply_chat_template(conversations,
#                                                             return_tensors='pt', padding=True)

#     return {
#         "image_embeddings": image_embeddings,
#         "input_ids": tokenized_conversations,
#         "attention_mask": torch.ones_like(tokenized_conversations),
#         "labels": tokenized_conversations.clone()
#     }

def prepare_dataset(examples):
    image_embeddings = []
    conversations = []

    for idx, conv in enumerate(examples['conversation']):
        image_embedding = torch.tensor(examples['image_embedding'][idx])
        dialogue_pairs = []

        for i in range(0, len(conv), 2):
            if i + 1 < len(conv):  # Ensure we have a pair
                human_msg = conv[i]['value'].replace('<image>', '').strip()
                gpt_msg = conv[i + 1]['value']

                dialogue = [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Given the following information, provide a detailed and accurate response:\n{human_msg}\n[An image is provided for this task.]\n"},
                    {"role": "assistant", "content": gpt_msg}
                ]

                dialogue_pairs.append(dialogue)
                image_embeddings.append(image_embedding)

        conversations.extend(dialogue_pairs)

    image_embeddings = torch.stack(image_embeddings)

    tokenized_conversations = tokenizer.apply_chat_template(conversations,
                                                            return_tensors='pt', padding=True)

    return {
        "image_embeddings": image_embeddings,
        "input_ids": tokenized_conversations,
        "attention_mask": torch.ones_like(tokenized_conversations),
        "labels": tokenized_conversations.clone()
    }

# Test the prepare_dataset function with a real training example
def test_prepare_dataset():
    # Get a batch of examples from the dataset
    batch_size = 1  # You can adjust this as needed
    sample_batch = hf_dataset[5:5+batch_size]

    print("Original conversations:")
    # for i, sample in enumerate(sample_batch):
    #     print(f"\nSample {i + 1}:")
    for message in sample_batch['conversation'][0]:
        print(f"{message['from']}: {message['value']}")

    # Process the sample batch
    result = prepare_dataset(sample_batch)

    # Print the structure of the result
    print("\nResult keys:", result.keys())
    print("Image embeddings shape:", result['image_embeddings'].shape)
    print("Input IDs shape:", result['input_ids'].shape)
    print("Attention mask shape:", result['attention_mask'].shape)
    print("Labels shape:", result['labels'].shape)

    for i in range(batch_size):
        decoded_input = tokenizer.decode(result['input_ids'][i])
        decoded_labels = tokenizer.decode(result['labels'][i])

        print(f"\nRestructured input for sample {i + 1}:")
        print(decoded_input)

        print(f"\nLabels for sample {i + 1}:")
        print(decoded_labels)

        # Optionally, you can print a more readable version of the labels
        print("\nReadable labels (non-padding tokens):")
        readable_labels = tokenizer.decode([token for token in result['labels'][i] if token != -100])
        print(readable_labels)

    # Optionally, you can print attention mask to see where it's applied
    print("\nAttention Mask:")
    print(result['attention_mask'][0])
# Run the test
test_prepare_dataset()


Original conversations:
human: <image>
What is the girl eating in the image?
gpt: The girl in the image is eating a dessert, which appears to be a graham cracker treat or a cookie sandwich.
human: Describe the girl's hair color and clothing.
gpt: The girl has blonde hair, and she is wearing a pink shirt.
human: What color is the plate that the dessert is on?
gpt: The dessert is on a green plate.
human: Is the girl looking at the camera or focusing on her dessert?
gpt: The girl is looking up at the camera while taking a bite of her dessert.
human: Where is the girl eating her dessert?
gpt: The girl is eating her dessert at the table.

Result keys: dict_keys(['image_embeddings', 'input_ids', 'attention_mask', 'labels'])
Image embeddings shape: torch.Size([5, 512])
Input IDs shape: torch.Size([5, 75])
Attention mask shape: torch.Size([5, 75])
Labels shape: torch.Size([5, 75])

Restructured input for sample 1:
<|system|> You are a helpful assistant.<|end|><|user|> Given the following infor

In [ ]:
# Apply tokenization and prepare the dataset
print("Applying tokenization and preparing the dataset...")


# def prepare_dataset(examples):
#     image_embeddings = torch.stack([torch.tensor(item) for item in examples['image_embedding']])

#     conversations = [
#         [{"role": "system", "content": "You are a helpful assistant."}] +
#         [{"role": "user" if msg['from'] == 'human' else "assistant", "content": msg['value']}
#          for msg in conv]
#         for conv in examples['conversation']
#     ]

#     tokenized_conversations = tokenizer.apply_chat_template(conversations,
#                                                              return_tensors='pt', padding=True)

#     return {
#         "image_embeddings": image_embeddings,
#         "input_ids": tokenized_conversations,
#         "attention_mask": torch.ones_like(tokenized_conversations),
#         "labels": tokenized_conversations.clone()
#     }


hf_dataset_mapped = hf_dataset.map(
    prepare_dataset,
    batched=True,
    remove_columns=hf_dataset.column_names,
    batch_size=1024  # Adjust based on your memory constraints
).with_format("torch")

# Split the dataset
train_test_split = hf_dataset_mapped.train_test_split(test_size=0.05)

# Create a DatasetDict
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

print(f"Train dataset size: {len(dataset_dict['train'])}")
print(f"Test dataset size: {len(dataset_dict['test'])}")

Applying tokenization and preparing the dataset...


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Train dataset size: 43134
Test dataset size: 2271


In [ ]:
# Example of accessing an item:
sample = dataset_dict['train'][0]
print(f"Input IDs shape: {len(sample['input_ids'])}")
print(f"Attention mask shape: {len(sample['attention_mask'])}")
print(f"Labels shape: {len(sample['labels'])}")

Input IDs shape: 678
Attention mask shape: 678
Labels shape: 678


### Projection Layer

In [ ]:
### Projection Layer
# class SimpleResBlock(nn.Module):
#     def __init__(self, input_dim, output_dim):
#         super().__init__()
#         self.pre_norm = nn.LayerNorm(input_dim)
#         self.proj = nn.Sequential(
#             nn.Linear(input_dim, output_dim),
#             nn.GELU(),
#             nn.Linear(output_dim, output_dim)
#         )

#     def forward(self, x):
#         x = self.pre_norm(x)
#         return x + self.proj(x)

# class ImageProjector(nn.Module):
#     def __init__(self, input_dim, output_dim):
#         super().__init__()
#         self.proj = nn.Linear(input_dim, output_dim)

#     def forward(self, x):
#         return self.proj(x)

In [ ]:
# import torch
# from torch.utils.data import Dataset, random_split

# class Phi3Dataset(Dataset):
#     def __init__(self, tokenized_data, projector, tokenizer):
#         self.data = tokenized_data
#         self.projector = projector
#         self.tokenizer = tokenizer

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         item = self.data[idx]
#         image_embedding = item['image_embedding']
#         conversation = item['tokenized_conversation']

#         projected_image = self.projector(image_embedding.unsqueeze(0)).squeeze(0)

#         # Combine projected image and conversation
#         combined_input = torch.cat([projected_image, conversation.squeeze(0)])

#         # Create attention mask
#         attention_mask = torch.ones_like(combined_input)

#         # Prepare labels (shift right, set first token to -100)
#         labels = torch.cat([-100 * torch.ones(projected_image.shape[0], dtype=torch.long), conversation.squeeze(0)[:-1]])

#         return {
#             "input_ids": combined_input,
#             "attention_mask": attention_mask,
#             "labels": labels
#         }

# # Usage:
# image_embedding_dim = tokenized_data[0]['image_embedding'].shape[-1]
# projection_dim = 1024  # Adjust as needed
# projector = SimpleResBlock(image_embedding_dim, projection_dim)
# full_dataset = Phi3Dataset(tokenized_data, projector, tokenizer)

# # Split the dataset
# train_size = int(0.9 * len(full_dataset))
# test_size = len(full_dataset) - train_size
# train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# print(f"Train dataset size: {len(train_dataset)}")
# print(f"Test dataset size: {len(test_dataset)}")

# # Example of accessing an item:
# sample = train_dataset[0]
# print(f"Input IDs shape: {sample['input_ids'].shape}")
# print(f"Attention mask shape: {sample['attention_mask'].shape}")
# print(f"Labels shape: {sample['labels'].shape}")


### QLoRA set up

In [ ]:
# new_model = "ms-phi3-custom"
lora_r = 16 #32 #64
lora_alpha = 16
lora_dropout = 0.05
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./multimodal-phi3_5-mini-instruct-llava_adapter"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 8
per_device_eval_batch_size = 4
gradient_accumulation_steps = 4
gradient_checkpointing = True
max_grad_norm = 1.0 #0.3
learning_rate = 5e-4
weight_decay = 0.0 #0.001
optim = "adamw_torch" #"paged_adamw_32bit"
lr_scheduler_type = "linear" #"constant"
max_steps = 12 #-1
warmup_ratio = 0.1 #0.03
group_by_length = True
save_steps = 5
logging_steps = 10
eval_steps = 10 # Evaluate every 25 steps
max_seq_length = 256
packing = False
device_map = {"": 0}
hf_adapter_repo="sayanbanerjee32/multimodal-phi3_5-mini-instruct-llava_adapter"

In [ ]:
# import torch
# from transformers import PreTrainedModel

# class Phi3WithProjector(PreTrainedModel):
#     supports_gradient_checkpointing = True

#     def __init__(self, phi3_model, projector):
#         super().__init__(phi3_model.config)
#         self.phi3 = phi3_model
#         self.projector = projector

#     def forward(self, input_ids=None, attention_mask=None, image_embeddings=None, labels=None, **kwargs):
#         device = next(self.parameters()).device

#         if image_embeddings is not None:
#             image_embeddings = image_embeddings.to(device)
#             projected_images = self.projector(image_embeddings)
#             projected_images = projected_images.unsqueeze(1)

#             if 'inputs_embeds' in kwargs and kwargs['inputs_embeds'] is not None:
#                 inputs_embeds = kwargs['inputs_embeds']
#                 inputs_embeds = torch.cat([projected_images, inputs_embeds], dim=1)
#                 kwargs['inputs_embeds'] = inputs_embeds
#             elif input_ids is not None:
#                 inputs_embeds = self.get_input_embeddings()(input_ids.to(device))
#                 inputs_embeds = torch.cat([projected_images, inputs_embeds], dim=1)
#                 kwargs['inputs_embeds'] = inputs_embeds
#                 input_ids = None  # Set to None to avoid conflict

#             if attention_mask is not None:
#                 attention_mask = torch.cat([torch.ones(image_embeddings.size(0), 1, device=device), attention_mask.to(device)], dim=1)
#             else:
#                 attention_mask = torch.ones(image_embeddings.size(0), inputs_embeds.size(1), device=device)

#             if labels is not None:
#                 # Adjust labels to match the new sequence length
#                 labels = torch.cat([torch.full((labels.size(0), 1), -100, device=device), labels], dim=1)

#         if labels is not None:
#             labels = labels.to(device)

#         # Ensure attention_mask matches the sequence length
#         if 'inputs_embeds' in kwargs:
#             seq_length = kwargs['inputs_embeds'].size(1)
#         elif input_ids is not None:
#             seq_length = input_ids.size(1)
#         else:
#             raise ValueError("Either input_ids or inputs_embeds should be provided")

#         if attention_mask is not None and attention_mask.size(1) != seq_length:
#             attention_mask = attention_mask[:, :seq_length]

#         return self.phi3(input_ids=input_ids, attention_mask=attention_mask, labels=labels, **kwargs)

#     def prepare_inputs_for_generation(self, input_ids, past=None, attention_mask=None, **kwargs):
#         inputs = self.phi3.prepare_inputs_for_generation(input_ids, past=past, attention_mask=attention_mask, **kwargs)
#         if 'image_embeddings' in kwargs:
#             inputs['image_embeddings'] = kwargs['image_embeddings']

#             # Adjust attention_mask if it's present
#             if attention_mask is not None:
#                 inputs['attention_mask'] = torch.cat([torch.ones(attention_mask.size(0), 1, device=attention_mask.device), attention_mask], dim=1)

#             # Remove position_ids as they're not used
#             inputs.pop('position_ids', None)

#         return inputs

#     def get_input_embeddings(self):
#         return self.phi3.get_input_embeddings()

#     def set_input_embeddings(self, value):
#         self.phi3.set_input_embeddings(value)

#     def gradient_checkpointing_enable(self, **kwargs):
#         self.phi3.gradient_checkpointing_enable(**kwargs)

#     def gradient_checkpointing_disable(self):
#         self.phi3.gradient_checkpointing_disable()

#     def __getattr__(self, name):
#         try:
#             return super().__getattr__(name)
#         except AttributeError:
#             return getattr(self.phi3, name)

#     def generate(self, input_ids=None, attention_mask=None, image_embeddings=None, **kwargs):
#         if image_embeddings is not None:
#             kwargs['image_embeddings'] = image_embeddings

#         if attention_mask is not None and image_embeddings is not None:
#             # Add an extra attention mask token for the image embedding
#             attention_mask = torch.cat([torch.ones(attention_mask.size(0), 1, device=attention_mask.device), attention_mask], dim=1)

#         return super().generate(input_ids=input_ids, attention_mask=attention_mask, **kwargs)

In [ ]:
# # import os
# # import torch
# from transformers import PreTrainedModel

# class ImageProjector(torch.nn.Module):
#     def __init__(self, input_dim, output_dim):
#         super().__init__()
#         self.linear = torch.nn.Linear(input_dim, output_dim)

#     def forward(self, x):
#         return self.linear(x)

# class Phi3WithProjector(PreTrainedModel):
#     supports_gradient_checkpointing = True

#     def __init__(self, phi3_model, projector):
#         super().__init__(phi3_model.config)
#         self.phi3 = phi3_model
#         self.projector = projector

#     @classmethod
#     def from_pretrained(cls, pretrained_model_name_or_path, *model_args, **kwargs):
#         # Load the base Phi-3 model
#         phi3_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path, *model_args, **kwargs)

#         # Load the projector weights
#         projector_path = os.path.join(pretrained_model_name_or_path, "image_projector.pth")
#         if os.path.exists(projector_path):
#             projector_state_dict = torch.load(projector_path, map_location=phi3_model.device)

#             # Check if the state dict has the expected structure
#             if 'linear.weight' in projector_state_dict:
#                 input_dim = projector_state_dict['linear.weight'].size(1)
#                 output_dim = projector_state_dict['linear.weight'].size(0)
#             else:
#                 # If not, try to infer dimensions from the first layer's weight
#                 first_key = next(iter(projector_state_dict))
#                 input_dim = projector_state_dict[first_key].size(1)
#                 output_dim = phi3_model.config.hidden_size  # Assuming this is the correct output dimension

#             projector = ImageProjector(input_dim, output_dim)

#             # Try to load the state dict, ignoring mismatched keys
#             projector.load_state_dict(projector_state_dict, strict=False)
#             print(f"Loaded projector with input_dim={input_dim}, output_dim={output_dim}")
#         else:
#             print(f"Projector weights not found at {projector_path}. Initializing with default dimensions.")
#             input_dim = 512  # Default CLIP embedding size
#             output_dim = phi3_model.config.hidden_size
#             projector = ImageProjector(input_dim, output_dim)

#         # Create and return the Phi3WithProjector instance
#         model = cls(phi3_model, projector)
#         return model

#     def save_pretrained(self, save_directory):
#         # Save the base model
#         self.phi3.save_pretrained(save_directory)

#         # Save the projector weights
#         projector_path = os.path.join(save_directory, "image_projector.pth")
#         torch.save(self.projector.state_dict(), projector_path)

#         # Save the config
#         self.config.save_pretrained(save_directory)

#     def forward(self, input_ids=None, attention_mask=None, image_embeddings=None, labels=None, **kwargs):
#         device = next(self.parameters()).device

#         if image_embeddings is not None:
#             image_embeddings = image_embeddings.to(device)
#             projected_images = self.projector(image_embeddings)
#             projected_images = projected_images.unsqueeze(1)

#             if 'inputs_embeds' in kwargs and kwargs['inputs_embeds'] is not None:
#                 inputs_embeds = kwargs['inputs_embeds']
#                 inputs_embeds = torch.cat([projected_images, inputs_embeds], dim=1)
#                 kwargs['inputs_embeds'] = inputs_embeds
#             elif input_ids is not None:
#                 inputs_embeds = self.get_input_embeddings()(input_ids.to(device))
#                 inputs_embeds = torch.cat([projected_images, inputs_embeds], dim=1)
#                 kwargs['inputs_embeds'] = inputs_embeds
#                 input_ids = None  # Set to None to avoid conflict

#             if attention_mask is not None:
#                 attention_mask = torch.cat([torch.ones(image_embeddings.size(0), 1, device=device), attention_mask.to(device)], dim=1)
#             else:
#                 attention_mask = torch.ones(image_embeddings.size(0), inputs_embeds.size(1), device=device)

#             if labels is not None:
#                 # Adjust labels to match the new sequence length
#                 labels = torch.cat([torch.full((labels.size(0), 1), -100, device=device), labels], dim=1)

#         if labels is not None:
#             labels = labels.to(device)

#         # Ensure attention_mask matches the sequence length
#         if 'inputs_embeds' in kwargs:
#             seq_length = kwargs['inputs_embeds'].size(1)
#         elif input_ids is not None:
#             seq_length = input_ids.size(1)
#         else:
#             raise ValueError("Either input_ids or inputs_embeds should be provided")

#         if attention_mask is not None and attention_mask.size(1) != seq_length:
#             attention_mask = attention_mask[:, :seq_length]

#         return self.phi3(input_ids=input_ids, attention_mask=attention_mask, labels=labels, **kwargs)

#     def prepare_inputs_for_generation(self, input_ids, past=None, attention_mask=None, **kwargs):
#         inputs = self.phi3.prepare_inputs_for_generation(input_ids, past=past, attention_mask=attention_mask, **kwargs)
#         if 'image_embeddings' in kwargs:
#             inputs['image_embeddings'] = kwargs['image_embeddings']

#             # Adjust attention_mask if it's present
#             if attention_mask is not None:
#                 inputs['attention_mask'] = torch.cat([torch.ones(attention_mask.size(0), 1, device=attention_mask.device), attention_mask], dim=1)

#             # Remove position_ids as they're not used
#             inputs.pop('position_ids', None)

#         return inputs

#     def get_input_embeddings(self):
#         return self.phi3.get_input_embeddings()

#     def set_input_embeddings(self, value):
#         self.phi3.set_input_embeddings(value)

#     def gradient_checkpointing_enable(self, **kwargs):
#         self.phi3.gradient_checkpointing_enable(**kwargs)

#     def gradient_checkpointing_disable(self):
#         self.phi3.gradient_checkpointing_disable()

#     def __getattr__(self, name):
#         try:
#             return super().__getattr__(name)
#         except AttributeError:
#             return getattr(self.phi3, name)

#     def generate(self, input_ids=None, attention_mask=None, image_embeddings=None, **kwargs):
#         if image_embeddings is not None:
#             kwargs['image_embeddings'] = image_embeddings

#         if attention_mask is not None and image_embeddings is not None:
#             # Add an extra attention mask token for the image embedding
#             attention_mask = torch.cat([torch.ones(attention_mask.size(0), 1, device=attention_mask.device), attention_mask], dim=1)

#         return super().generate(input_ids=input_ids, attention_mask=attention_mask, **kwargs)

In [ ]:

class ImageProjector(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=1024):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.GELU()
        self.layer2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.05)

        # Store initial weights for both layers
        self.register_buffer('initial_weights1', self.layer1.weight.data.clone())
        self.register_buffer('initial_weights2', self.layer2.weight.data.clone())

    def forward(self, x):
        # Print dtypes
        # print(f"Input dtype: {x.dtype}")
        # print(f"Layer1 weight dtype: {self.layer1.weight.dtype}")
        # print(f"Layer1 bias dtype: {self.layer1.bias.dtype}")
        # print(f"Layer2 weight dtype: {self.layer2.weight.dtype}")
        # print(f"Layer2 bias dtype: {self.layer2.bias.dtype}")
        x = self.layer1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.layer2(x)
        return x

    def get_weight_change(self):
        current_weights1 = self.layer1.weight.data
        current_weights2 = self.layer2.weight.data

        # Ensure all tensors are on the same device
        device = current_weights1.device
        initial_weights1 = self.initial_weights1.to(device)
        initial_weights2 = self.initial_weights2.to(device)

        weight_diff1 = torch.norm(current_weights1 - initial_weights1).item()
        weight_diff2 = torch.norm(current_weights2 - initial_weights2).item()
        return weight_diff1 + weight_diff2  # Total weight change across both layers

class Phi3WithProjector(PreTrainedModel):
    supports_gradient_checkpointing = True
    _supports_sdpa = True # Add this line

    def __init__(self, phi3_model, projector, debug=False):
        super().__init__(phi3_model.config)
        self.phi3 = phi3_model
        self.projector = projector
        self.debug = debug

    def debug_print(self, *args, **kwargs):
        if self.debug:
            print(*args, **kwargs)

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path, *model_args, debug=False, **kwargs):
        # Load the base Phi-3 model
        phi3_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path, *model_args, **kwargs)

        # Determine if it's a local path or a Hugging Face model ID
        is_local = os.path.isdir(pretrained_model_name_or_path)

        if is_local:
            projector_path = os.path.join(pretrained_model_name_or_path, "image_projector.pth")
        else:
            try:
                # Try to download the projector weights from the Hugging Face Hub
                projector_path = hf_hub_download(repo_id=pretrained_model_name_or_path, filename="image_projector.pth")
            except Exception as e:
                print(f"Failed to download projector weights: {e}")
                projector_path = None

        if projector_path and os.path.exists(projector_path):
            projector_state_dict = torch.load(projector_path, map_location=phi3_model.device)
            # Check if the state dict has the expected structure
            if 'linear.weight' in projector_state_dict:
                input_dim = projector_state_dict['linear.weight'].size(1)
                output_dim = projector_state_dict['linear.weight'].size(0)
            else:
                # If not, try to infer dimensions from the first layer's weight
                first_key = next(iter(projector_state_dict))
                input_dim = projector_state_dict[first_key].size(1)
                output_dim = phi3_model.config.hidden_size  # Assuming this is the correct output dimension

            projector = ImageProjector(input_dim, output_dim)

            # Convert projector weights and biases to the same dtype as the main model
            target_dtype = kwargs.get('torch_dtype', torch.float32)
            projector_state_dict = {k: v.to(target_dtype) for k, v in projector_state_dict.items()}

            # Load the state dict with converted weights and biases
            projector.load_state_dict(projector_state_dict, strict=False)

            # Ensure all parameters (including biases) are in the correct dtype
            for param in projector.parameters():
                param.data = param.data.to(target_dtype)

            print(f"Loaded projector with input_dim={input_dim}, output_dim={output_dim}, dtype={target_dtype}")
        else:
            print(f"Projector weights not found. Initializing with default dimensions.")
            input_dim = 512  # Default CLIP embedding size
            output_dim = phi3_model.config.hidden_size
            target_dtype = kwargs.get('torch_dtype', torch.float32)
            projector = ImageProjector(input_dim, output_dim)
            # Ensure all parameters (including biases) are in the correct dtype
            for param in projector.parameters():
                param.data = param.data.to(target_dtype)

        # Move the projector to the same device as phi3_model
        projector = projector.to(phi3_model.device)

        # Create and return the Phi3WithProjector instance
        model = cls(phi3_model, projector, debug=debug)
        return model


    def save_pretrained(self, save_directory):
        print(f"Saving model to {save_directory}")

        # Save the base model
        self.phi3.save_pretrained(save_directory)

        # Save the projector weights
        projector_path = os.path.join(save_directory, "image_projector.pth")
        projector_state = self.projector.state_dict()
        print(f"Projector weights stats before saving:")
        for name, param in projector_state.items():
            print(f"  {name}: mean={param.mean().item():.4f}, std={param.std().item():.4f}")
        torch.save(projector_state, projector_path)

        # Save the config
        self.config.save_pretrained(save_directory)

        print(f"Model saved successfully to {save_directory}")

    def forward(self, input_ids=None, attention_mask=None, labels=None, image_embeddings=None, past_key_values=None, **kwargs):
        device = next(self.parameters()).device

        if image_embeddings is not None:
            projected_embeddings = self.projector(image_embeddings)
            # Ensure projected_embeddings requires grad
            if not projected_embeddings.requires_grad:
                projected_embeddings.requires_grad_(True)
            projected_embeddings = projected_embeddings.unsqueeze(1)
            self.debug_print(f"forward projected_embeddings: {projected_embeddings.size()}")

            if past_key_values is None:  # This is the first forward pass
                self.debug_print(f"forward before: {attention_mask.size() if attention_mask is not None else None}")
                if 'inputs_embeds' in kwargs and kwargs['inputs_embeds'] is not None:
                    inputs_embeds = kwargs['inputs_embeds']
                    self.debug_print(f"forward before inputs_embeds: {inputs_embeds.size()}")
                    inputs_embeds = torch.cat([projected_embeddings, inputs_embeds], dim=1)
                    kwargs['inputs_embeds'] = inputs_embeds
                    self.debug_print(f"forward after inputs_embeds: {inputs_embeds.size()}")
                elif input_ids is not None:
                    self.debug_print(f"forward input_ids: {input_ids.size()}")
                    inputs_embeds = self.get_input_embeddings()(input_ids.to(device))
                    self.debug_print(f"forward before inputs_embeds: {inputs_embeds.size()}")
                    inputs_embeds = torch.cat([projected_embeddings, inputs_embeds], dim=1)
                    self.debug_print(f"forward after inputs_embeds: {inputs_embeds.size()}")
                    kwargs['inputs_embeds'] = inputs_embeds
                    input_ids = None  # Set to None to avoid conflict

                if attention_mask is not None:
                    attention_mask = torch.cat([torch.ones(image_embeddings.size(0), 1, device=device), attention_mask.to(device)], dim=1)
                else:
                    attention_mask = torch.ones(image_embeddings.size(0), inputs_embeds.size(1), device=device)

                if labels is not None:
                    # Adjust labels to match the new sequence length
                    labels = torch.cat([torch.full((labels.size(0), 1), -100, device=device), labels], dim=1)

        if labels is not None:
            labels = labels.to(device)

        # Determine sequence length
        if 'inputs_embeds' in kwargs and kwargs['inputs_embeds'] is not None:
            seq_length = kwargs['inputs_embeds'].size(1)
        elif input_ids is not None:
            seq_length = input_ids.size(1)
        else:
            seq_length = attention_mask.size(1) if attention_mask is not None else None

        if seq_length is None:
            raise ValueError("Unable to determine sequence length. Provide either input_ids, inputs_embeds, or attention_mask.")

        # Ensure attention_mask matches the sequence length
        if attention_mask is not None:
            attention_mask = attention_mask[:, :seq_length]

        self.debug_print(f"forward final: input_ids shape: {input_ids.shape if input_ids is not None else None}")
        self.debug_print(f"forward final: attention_mask shape: {attention_mask.shape if attention_mask is not None else None}")
        self.debug_print(f"forward final: inputs_embeds shape: {kwargs.get('inputs_embeds', {}).shape if kwargs.get('inputs_embeds') is not None else None}")

        return self.phi3(input_ids=input_ids, attention_mask=attention_mask, labels=labels, past_key_values=past_key_values, **kwargs)

    def prepare_inputs_for_generation(self, input_ids, past=None, attention_mask=None, **kwargs):
        inputs = self.phi3.prepare_inputs_for_generation(input_ids, past=past, attention_mask=attention_mask, **kwargs)

        if 'image_embeddings' in kwargs:
            inputs['image_embeddings'] = kwargs['image_embeddings']

            if past is None:  # First forward pass
                # Adjust attention_mask to account for the image token
                if attention_mask is not None:
                    inputs['attention_mask'] = torch.cat([torch.ones((attention_mask.size(0), 1), device=attention_mask.device), attention_mask], dim=1)
            else:  # Subsequent passes
                # Ensure attention_mask matches the current sequence length
                if attention_mask is not None:
                    current_seq_length = past[0][0].size(2) + 1  # past key's sequence length + 1 for the new token
                    inputs['attention_mask'] = attention_mask[:, :current_seq_length]

            inputs.pop('position_ids', None)

        # Safe printing of shapes
        self.debug_print(f"prepare_inputs_for_generation: input_ids shape: {inputs['input_ids'].shape if 'input_ids' in inputs else None}")
        self.debug_print(f"prepare_inputs_for_generation: attention_mask shape: {inputs['attention_mask'].shape if 'attention_mask' in inputs else None}")
        self.debug_print(f"prepare_inputs_for_generation: inputs_embeds shape: {inputs.get('inputs_embeds', {}).shape if inputs.get('inputs_embeds') is not None else None}")

        return inputs

    def get_input_embeddings(self):
        return self.phi3.get_input_embeddings()

    def set_input_embeddings(self, value):
        self.phi3.set_input_embeddings(value)

    def gradient_checkpointing_enable(self, **kwargs):
        self.phi3.gradient_checkpointing_enable(**kwargs)

    def gradient_checkpointing_disable(self):
        self.phi3.gradient_checkpointing_disable()

    def __getattr__(self, name):
        try:
            return super().__getattr__(name)
        except AttributeError:
            return getattr(self.phi3, name)

    def generate(self, input_ids=None, attention_mask=None, image_embeddings=None, **kwargs):
        if image_embeddings is not None:
            kwargs['image_embeddings'] = image_embeddings
            self.debug_print(f"generate input_ids: {input_ids.size()}")
            self.debug_print(f"generate image_embedding: {image_embeddings.size()}")

        if attention_mask is not None and image_embeddings is not None:
            # Add an extra attention mask token for the image embedding
            self.debug_print(f"generate before: {attention_mask.size()}")
            attention_mask = torch.cat([torch.ones(attention_mask.size(0), 1, device=attention_mask.device), attention_mask], dim=1)
            self.debug_print(f"generate after: {attention_mask.size()}")

        return super().generate(input_ids=input_ids, attention_mask=attention_mask, **kwargs)

In [ ]:
# Define the path in Google Drive where you want to save the checkpoints
gdrive_checkpoint_dir = "/content/drive/MyDrive/multimodal_llm/phi-3_5/checkpoints"

# Ensure the directory exists
os.makedirs(gdrive_checkpoint_dir, exist_ok=True)
import dataclasses

class SaveLatestCheckpointCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        if state.is_world_process_zero:
            checkpoint_dir = os.path.join(gdrive_checkpoint_dir, f"checkpoint-{state.global_step}")

            # Save the model and tokenizer
            kwargs["model"].save_pretrained(checkpoint_dir)
            kwargs["tokenizer"].save_pretrained(checkpoint_dir)

            # Log and save the projector
            projector = kwargs["model"].base_model.model.projector
            weight_change = projector.get_weight_change()
            print(f"Projector weight change at step {state.global_step}: {weight_change}")

            # # Add gradient checking here
            # for name, param in projector.named_parameters():
            #     if param.grad is None:
            #         print(f"No gradient for {name}")
            #     else:
            #         print(f"Gradient norm for {name}: {param.grad.norm().item()}")

            projector_path = os.path.join(checkpoint_dir, "image_projector.pth")
            torch.save(projector.state_dict(), projector_path)

            # Explicitly save the trainer state
            trainer_state_path = os.path.join(checkpoint_dir, "trainer_state.json")
            state_dict = dataclasses.asdict(state)
            with open(trainer_state_path, "w") as f:
                json.dump(state_dict, f, indent=2)

            # Remove previous checkpoint
            prev_checkpoint = os.path.join(gdrive_checkpoint_dir, f"checkpoint-{state.global_step - args.save_steps}")
            if os.path.exists(prev_checkpoint):
                import shutil
                shutil.rmtree(prev_checkpoint)

class SaveQLoraAdapterCallback(TrainerCallback):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def on_save(self, args, state, control, **kwargs):
        checkpoint_dir = os.path.join(args.output_dir, f"checkpoint-{state.global_step}")
        os.makedirs(checkpoint_dir, exist_ok=True)

        # Save LoRA weights
        lora_state_dict = {}
        for name, module in self.model.base_model.model.phi3.named_modules():
            if isinstance(module, lora.Linear4bit):
                if hasattr(module, 'lora_A'):
                    lora_state_dict[f"{name}.lora_A.weight"] = module.lora_A.default.weight.data.cpu()
                    lora_state_dict[f"{name}.lora_B.weight"] = module.lora_B.default.weight.data.cpu()
                    lora_state_dict[f"{name}.scaling"] = module.scaling

                # Save lora_embedding_A and lora_embedding_B if they exist and are not empty
                if hasattr(module, 'lora_embedding_A') and module.lora_embedding_A:
                    lora_state_dict[f"{name}.lora_embedding_A"] = {k: v.cpu() for k, v in module.lora_embedding_A.items()}
                if hasattr(module, 'lora_embedding_B') and module.lora_embedding_B:
                    lora_state_dict[f"{name}.lora_embedding_B"] = {k: v.cpu() for k, v in module.lora_embedding_B.items()}

        torch.save(lora_state_dict, os.path.join(checkpoint_dir, "lora_weights.pt"))

        # Save projector weights using the built-in method
        self.model.base_model.model.save_pretrained(checkpoint_dir)

        print(f"Saved checkpoint to {checkpoint_dir}")

# Function to get the latest checkpoint
def get_latest_checkpoint(checkpoint_dir):
    checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith('checkpoint-')]
    if not checkpoints:
        return None
    latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[1]))
    return os.path.join(checkpoint_dir, latest_checkpoint)

In [ ]:
if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
#   attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
#   attn_implementation = 'sdpa'

# print(attn_implementation)
print(compute_dtype)

torch.bfloat16


In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# # Load the model again for quantization
# ### Download Phi-3 model
# phi3_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     trust_remote_code=True,
#     quantization_config=bnb_config,
#     device_map=device_map,
#     torch_dtype=compute_dtype,
#     # attn_implementation=attn_implementation
# )

# print(phi3_model)

In [ ]:
# # Initialize the projector
# image_embedding_dim = len(hf_dataset[0]['image_embedding'])
# projection_dim = phi3_model.config.hidden_size  # Get dimension from the model
# projector = ImageProjector(image_embedding_dim, projection_dim).to(device)

# # Combine Phi-3 with the projector
# model = Phi3WithProjector(phi3_model, projector)

# Get the latest checkpoint
latest_checkpoint = get_latest_checkpoint(gdrive_checkpoint_dir)
eval_first = False
if latest_checkpoint:
    print(f"Loading model from checkpoint: {latest_checkpoint}")
    model = Phi3WithProjector.from_pretrained(
        latest_checkpoint,
        trust_remote_code=True,
        quantization_config=bnb_config,
        device_map=device_map,
        torch_dtype=compute_dtype,
        attn_implementation='eager'
    )
    eval_first = True
else:
    print("No checkpoint found. Starting training from scratch.")
    phi3_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        quantization_config=bnb_config,
        device_map=device_map,
        torch_dtype=compute_dtype,
        attn_implementation='eager'
    )
    image_embedding_dim = len(hf_dataset[0]['image_embedding'])
    projection_dim = phi3_model.config.hidden_size
    projector = ImageProjector(image_embedding_dim, projection_dim).to(device)
    model = Phi3WithProjector(phi3_model, projector)

# Prepare the model for k-bit training
model = prepare_model_for_kbit_training(model)

No checkpoint found. Starting training from scratch.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [ ]:
print(compute_dtype) , print(model)

torch.float16
Phi3WithProjector(
  (phi3): Phi3ForCausalLM(
    (model): Phi3Model(
      (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
      (embed_dropout): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0-31): 32 x Phi3DecoderLayer(
          (self_attn): Phi3Attention(
            (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
            (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
            (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
          )
          (mlp): Phi3MLP(
            (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
            (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
            (activation_fn): SiLU()
          )
          (input_layernorm): Phi3RMSNorm()
          (resid_attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
          (post_attention_layernorm): Ph

(None, None)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [ ]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"],
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

trainable params: 8912896 || all params: 2021727232 || trainable%: 0.44085551497384196


In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3WithProjector(
      (phi3): Phi3ForCausalLM(
        (model): Phi3Model(
          (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
          (embed_dropout): Dropout(p=0.0, inplace=False)
          (layers): ModuleList(
            (0-31): 32 x Phi3DecoderLayer(
              (self_attn): Phi3Attention(
                (o_proj): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=3072, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=3072, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
           

In [ ]:
def verify_projector_weight_changes(loaded_state_dict, initial_state_dict):
    for key in loaded_state_dict:
        if torch.norm(loaded_state_dict[key] - initial_state_dict[key]).item() > 0:
            print(f"Weights changed for {key}")
            return True
    print("WARNING: No weight changes detected in the projector!")
    return False

# Use this when loading the model
initial_projector_state = copy.deepcopy(model.base_model.model.projector.state_dict())

### Training

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size  = per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    eval_strategy="steps",
    eval_steps=eval_steps, # Evaluate every 25 steps
    # Add these new arguments
    do_eval=True,
    eval_delay=0,  # Start evaluation immediately
    # Enable gradient checkpointing
    gradient_checkpointing=gradient_checkpointing,
    # Disable data parallelism if not needed
    ddp_find_unused_parameters=False,
    save_total_limit=1,  # Keep only the latest checkpoint
    hub_model_id = hf_adapter_repo
)



In [ ]:
# Custom data collator to handle pre-tokenized inputs
def custom_data_collator(features):
    batch = {k: [d[k] for d in features] for k in features[0].keys()}

    # Stack image embeddings
    batch['image_embeddings'] = torch.stack(batch['image_embeddings'])

    # Pad the sequences
    batch['input_ids'] = torch.nn.utils.rnn.pad_sequence(batch['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id)
    batch['attention_mask'] = torch.nn.utils.rnn.pad_sequence(batch['attention_mask'], batch_first=True, padding_value=0)
    batch['labels'] = torch.nn.utils.rnn.pad_sequence(batch['labels'], batch_first=True, padding_value=-100)

    return batch

# Function to select a random subset of the dataset
def select_subset(dataset, fraction=0.05):
    num_samples = int(len(dataset) * fraction)
    indices = random.sample(range(len(dataset)), num_samples)
    return dataset.select(indices)

# Select 5% of the training and test datasets
small_train_dataset = select_subset(dataset_dict['train'], fraction=0.1)
small_test_dataset = select_subset(dataset_dict['test'], fraction=0.1)

# Create a new DatasetDict with the smaller datasets
small_dataset_dict = DatasetDict({
    'train': small_train_dataset,
    'test': small_test_dataset
})

print(f"Small train dataset size: {len(small_dataset_dict['train'])}")
print(f"Small test dataset size: {len(small_dataset_dict['test'])}")

Small train dataset size: 4313
Small test dataset size: 227


In [ ]:
# Before initializing the SFTTrainer
model.projector.train()  # Set projector to training mode
for param in model.projector.parameters():
    param.requires_grad = True

def hook_fn(module, grad_input, grad_output):
    print(f"Gradient for {module.__class__.__name__}:")
    for idx, grad in enumerate(grad_input):
        if grad is not None:
            print(f"  Input gradient {idx}: {grad.norm().item()}")

# # Apply the hook to projector layers
# model.projector.layer1.register_backward_hook(hook_fn)
# model.projector.layer2.register_backward_hook(hook_fn)

In [ ]:
# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    # train_dataset=dataset_dict['train'],
    # eval_dataset=dataset_dict['test'],
    train_dataset=small_dataset_dict['train'],
    eval_dataset=small_dataset_dict['test'],
    tokenizer=tokenizer,
    data_collator=custom_data_collator,
    peft_config=lora_config,
    max_seq_length=max_seq_length,
    packing=packing,
    callbacks=[SaveLatestCheckpointCallback(), SaveQLoraAdapterCallback(model)],  # Add the custom callback
)

# Perform initial evaluation
if eval_first:
    print("Performing initial evaluation...")
    eval_results = trainer.evaluate()
    print(f"Initial evaluation results: {eval_results}")
    # Start or resume training
    trainer.train(resume_from_checkpoint=latest_checkpoint)
else:
    # Start training
    trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in

Step,Training Loss,Validation Loss
10,3.312700,0.371034


Projector weight change at step 5: 1.9239861369132996
Saving model to ./multimodal-phi3_5-mini-instruct-llava_adapter/checkpoint-5
Projector weights stats before saving:
  initial_weights1: mean=0.0000, std=0.0255
  initial_weights2: mean=-0.0000, std=0.0180
  layer1.weight: mean=0.0000, std=0.0255
  layer1.bias: mean=-0.0003, std=0.0256
  layer2.weight: mean=-0.0000, std=0.0181
  layer2.bias: mean=0.0002, std=0.0178
Model saved successfully to ./multimodal-phi3_5-mini-instruct-llava_adapter/checkpoint-5
Saved checkpoint to ./multimodal-phi3_5-mini-instruct-llava_adapter/checkpoint-5


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Projector weight change at step 10: 2.7955992817878723
Saving model to ./multimodal-phi3_5-mini-instruct-llava_adapter/checkpoint-10
Projector weights stats before saving:
  initial_weights1: mean=0.0000, std=0.0255
  initial_weights2: mean=-0.0000, std=0.0180
  layer1.weight: mean=0.0000, std=0.0255
  layer1.bias: mean=-0.0003, std=0.0256
  layer2.weight: mean=-0.0000, std=0.0181
  layer2.bias: mean=0.0002, std=0.0178
Model saved successfully to ./multimodal-phi3_5-mini-instruct-llava_adapter/checkpoint-10
Saved checkpoint to ./multimodal-phi3_5-mini-instruct-llava_adapter/checkpoint-10


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Projector weight change at step 12: 2.866250216960907
Saving model to ./multimodal-phi3_5-mini-instruct-llava_adapter/checkpoint-12
Projector weights stats before saving:
  initial_weights1: mean=0.0000, std=0.0255
  initial_weights2: mean=-0.0000, std=0.0180
  layer1.weight: mean=0.0000, std=0.0255
  layer1.bias: mean=-0.0003, std=0.0256
  layer2.weight: mean=-0.0000, std=0.0181
  layer2.bias: mean=0.0002, std=0.0178
Model saved successfully to ./multimodal-phi3_5-mini-instruct-llava_adapter/checkpoint-12
Saved checkpoint to ./multimodal-phi3_5-mini-instruct-llava_adapter/checkpoint-12


In [ ]:
# Save the fine-tuned model
trainer.save_model()
# trainer.model.save_pretrained(new_model)
final_model_path = os.path.join(gdrive_checkpoint_dir, "final_model")
trainer.model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)

('/content/drive/MyDrive/multimodal_llm/phi-3_5/checkpoints/final_model/tokenizer_config.json',
 '/content/drive/MyDrive/multimodal_llm/phi-3_5/checkpoints/final_model/special_tokens_map.json',
 '/content/drive/MyDrive/multimodal_llm/phi-3_5/checkpoints/final_model/tokenizer.model',
 '/content/drive/MyDrive/multimodal_llm/phi-3_5/checkpoints/final_model/added_tokens.json',
 '/content/drive/MyDrive/multimodal_llm/phi-3_5/checkpoints/final_model/tokenizer.json')

In [ ]:
torch.save(model.base_model.model.projector.state_dict(), final_model_path + '/image_projector.pth')
print(f"Projector saved to: {final_model_path}/image_projector.pth")

Projector saved to: /content/drive/MyDrive/multimodal_llm/phi-3_5/checkpoints/final_model/image_projector.pth


In [ ]:
# Save the projector
projector_path = '/content/drive/MyDrive/multimodal_llm/phi-3_5/image_projector.pth'
os.makedirs(os.path.dirname(projector_path), exist_ok=True)
torch.save(model.projector.state_dict(), projector_path)
print(f"Projector saved to: {projector_path}")

Projector saved to: /content/drive/MyDrive/multimodal_llm/phi-3_5/image_projector.pth


In [ ]:
loaded_projector_state = torch.load(projector_path)
verify_projector_weight_changes(loaded_projector_state, initial_projector_state)

Weights changed for layer1.weight


<ipython-input-37-9c63dfc99e55>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_projector_state = torch.load(projector_path)


True

In [ ]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/35.7M [00:00<?, ?B/s]

events.out.tfevents.1729267081.8ce57b37d7a3.775.0:   0%|          | 0.00/9.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sayanbanerjee32/multimodal-phi3_5-mini-instruct-llava_adapter/commit/ddf7ca1b7788294593e4b9b82f162dd13c29c667', commit_message='End of training', commit_description='', oid='ddf7ca1b7788294593e4b9b82f162dd13c29c667', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
!cp -r multimodal-phi3_5-mini-instruct-llava_adapter /content/drive/MyDrive/multimodal_llm/phi-3_5

In [ ]:
# Add this function after your CustomTextGenerator class
def evaluate_model(model, tokenizer, eval_dataset, device):
    model = model.to(torch.float16).to(device)
    model.eval()
    total_loss = 0
    num_batches = 0

    dataloader = DataLoader(eval_dataset, batch_size=2, collate_fn=custom_data_collator)

    with torch.no_grad():
        for batch in dataloader:
            batch = {
                'input_ids': batch['input_ids'].to(device).int(),
                'attention_mask': batch['attention_mask'].to(device).int(),
                'image_embeddings': batch['image_embeddings'].to(device).to(torch.float16),
                'labels': batch['labels'].to(device).int()
            }

            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                image_embeddings=batch['image_embeddings'],
                labels=batch['labels']
            )
            total_loss += outputs.loss.item()
            num_batches += 1

    return total_loss / num_batches if num_batches > 0 else 0

# Add these lines before merging the model
print("Evaluating model before merging...")
pre_merge_loss = evaluate_model(model, tokenizer, small_dataset_dict['test'], device)
print(f"Pre-merge loss: {pre_merge_loss}")

Evaluating model before merging...
Pre-merge loss: 0.3541485089481923


## sample inference code

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Create a custom text generation class
class CustomTextGenerator:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def generate(self, input_text, image_embedding, **generate_kwargs):
        # Tokenize the input text
        inputs = self.tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
        input_ids = inputs["input_ids"].to(self.model.device)
        attention_mask = inputs["attention_mask"].to(self.model.device)

        # Ensure image_embedding is a tensor and move it to the correct device
        if not isinstance(image_embedding, torch.Tensor):
            image_embedding = torch.tensor(image_embedding)
        # image_embedding = image_embedding.to(self.model.device).unsqueeze(0)  # Add batch dimension
        image_embedding = image_embedding.to(self.model.device).to(next(self.model.parameters()).dtype)

        # Generate text
        outputs = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # image_embeddings=image_embedding,
            image_embeddings=image_embedding.unsqueeze(0),  # Add batch dimension
            **generate_kwargs
        )

        # Decode the generated text
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return generated_text

In [ ]:
# Get a sample from the validation set
sample = dataset_dict['test'][1]
image_embedding = sample['image_embeddings']


def get_first_user_input(decoded_text):
    # Find the position of the first <|assistant|> tag
    assistant_pos = decoded_text.find('<|assistant|>')

    # If <|assistant|> is found, truncate the text
    if assistant_pos != -1:
        return decoded_text[:assistant_pos].strip()
    else:
        return decoded_text.strip()

# Initialize the custom text generator
generator = CustomTextGenerator(model=model, tokenizer=tokenizer)
# Decode the input_ids
full_text = tokenizer.decode(sample['input_ids'], skip_special_tokens=False)

# Extract only the first user input
input_text = get_first_user_input(full_text) + '<|assistant|>'
print(input_text)

<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>

In [ ]:
def test_inference(model, tokenizer, image_embedding):
    # Log projector weights before inference
    projector = model.projector
    weight_change = projector.get_weight_change()
    print(f"Projector weight change: {weight_change}")
    # Ensure the model is in the correct precision (e.g., float16)
    model = model.to(next(model.parameters()).dtype)

    generator = CustomTextGenerator(model=model, tokenizer=tokenizer)
    input_text = """<|system|> You are a helpful assistant.<|end|><|user|> Given the following information, provide a detailed and accurate response:
Describe the image in detail.
[An image is provided for this task.]
<|end|><|assistant|>
"""
    generated_text = generator.generate(
        input_text,
        image_embedding=image_embedding,
        max_new_tokens=150,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.8,
        top_k=40,
        top_p=0.9,
        repetition_penalty=1.2,
        no_repeat_ngram_size=3,
    )
    return generated_text



In [ ]:
# Generate text
generated_text = generator.generate(
    input_text,
    image_embedding=image_embedding,
    # max_length=200,
    # num_return_sequences=1,
    # do_sample=True,
    # temperature=0.7,
    # top_k=50,
    # top_p=0.95,
    max_new_tokens=150,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.8,
    top_k=40,
    top_p=0.9,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3,
)

print("Input text:")
print(input_text)
print("\nGenerated text:")
print(generated_text)

Input text:
<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><un

In [ ]:
# Before merging
print("Testing inference before merging...")
pre_merge_output = test_inference(model, tokenizer, sample['image_embeddings'])
print(pre_merge_output)


Testing inference before merging...
Projector weight change: 2.86572265625
You are a helpful assistant. Given the following information, provide a detailed and accurate response:
Describe the image in detail.
[An image is provided for this task.]
 1) The picture presents an artistic representation of "The Great Gatsby" by F Scott Fitzgerald; it illustrates one scene that depicts Jay Gatsbys extravagant mansion on West Egg during his lavish parties with numerous guests attending from different social backgrounds – rich aristocratic individuals contrasted against characters like Myrtle Wilson who aspires to rise above her lower-class status but ultimately fails due to societal limitations highlighting class struggles prevalent within American society as portrayed through literary fiction at its core - all whilst incorporating visual elements such symbolic color schemes (e.g., opulence represented via gold tones & red hues), light effects


### merge models and save in gdrive

In [ ]:
del trainer
del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from bitsandbytes.nn import LinearFP4
def load_model_with_lora_and_projector(checkpoint_path, device):
    is_cpu = device == "cpu"

    if is_cpu:
        model = Phi3WithProjector.from_pretrained(
            checkpoint_path,
            trust_remote_code=True,
            torch_dtype=torch.float32,
            low_cpu_mem_usage=True
        )
    else:
        model = Phi3WithProjector.from_pretrained(
            checkpoint_path,
            trust_remote_code=True,
            quantization_config=bnb_config,
            device_map=device,
            torch_dtype=torch.float16,
            attn_implementation='eager'
        )

    model = model.to(device)

    lora_state_dict = torch.load(os.path.join(checkpoint_path, "lora_weights.pt"), map_location=device)
    print(f"Total keys in lora_state_dict: {len(lora_state_dict)}")

    new_state_dict = {}
    scaling_factors_loaded = 0

    for name, module in model.phi3.named_modules():
        if isinstance(module, lora.Linear4bit):
            lora_A_key = f"{name}.lora_A.weight"
            lora_B_key = f"{name}.lora_B.weight"
            scaling_key = f"{name}.scaling"

            if lora_A_key in lora_state_dict and lora_B_key in lora_state_dict:
                new_state_dict[f"{name}.lora_A.default.weight"] = lora_state_dict[lora_A_key]
                new_state_dict[f"{name}.lora_B.default.weight"] = lora_state_dict[lora_B_key]

                if scaling_key in lora_state_dict:
                    module.scaling = lora_state_dict[scaling_key]
                    scaling_factors_loaded += 1
            else:
                print(f"Warning: LoRA weights for {name} not found in checkpoint")

    # Load the filtered state dict
    model.phi3.load_state_dict(new_state_dict, strict=False)

    print(f"Loaded LoRA weights: {len(new_state_dict)} / {len(lora_state_dict)}")
    print(f"Loaded scaling factors: {scaling_factors_loaded}")
    print(f"Total LoRA modules processed: {len(new_state_dict) // 2}")

    # Prepare the model for inference
    for name, module in model.named_modules():
        if isinstance(module, LinearFP4): # or isinstance(module, LinearFP8):
            print("Preparing for inference")
            module.prepare_for_inference()

    return model

In [ ]:
loaded_model = load_model_with_lora_and_projector("/content/drive/MyDrive/multimodal_llm/phi-3_5/multimodal-phi3_5-mini-instruct-llava_adapter/checkpoint-12",
                                                  device)
# print(loaded_model.projector.layer1.weight.dtype, loaded_model.projector.layer1.bias.dtype)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading adapter weights from /content/drive/MyDrive/multimodal_llm/phi-3_5/multimodal-phi3_5-mini-instruct-llava_adapter/checkpoint-12 led to unexpected keys not found in the model:  ['phi3.model.layers.0.mlp.down_proj.lora_A.default.weight', 'phi3.model.layers.0.mlp.down_proj.lora_B.default.weight', 'phi3.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'phi3.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'phi3.model.layers.1.mlp.down_proj.lora_A.default.weight', 'phi3.model.layers.1.mlp.down_proj.lora_B.default.weight', 'phi3.model.layers.1.self_attn.o_proj.lora_A.default.weight', 'phi3.model.layers.1.self_attn.o_proj.lora_B.default.weight', 'phi3.model.layers.10.mlp.down_proj.lora_A.default.weight', 'phi3.model.layers.10.mlp.down_proj.lora_B.default.weight', 'phi3.model.layers.10.self_attn.o_proj.lora_A.default.weight', 'phi3.model.layers.10.self_attn.o_proj.lora_B.default.weight', 'phi3.model.layers.11.mlp.down_proj.lora_A.default.weight', 'phi3.model.layers.11.mlp.

Loaded projector with input_dim=512, output_dim=3072, dtype=torch.float16


<ipython-input-30-3a5fbc10ea60>:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lora_state_dict = torch.load(os.path.join(checkpoint_path, "lora_weights.pt"), map_location

Total keys in lora_state_dict: 192
Loaded LoRA weights: 128 / 192
Loaded scaling factors: 64
Total LoRA modules processed: 64


In [ ]:
def evaluate_loaded_model(model, tokenizer, eval_dataset, device):
    model = model.to(device)
    model.eval()

    # # Print model parameter dtypes
    # for name, param in model.named_parameters():
    #     if 'projector' in name:
    #         print(f"{name} dtype: {param.dtype}")
    total_loss = 0
    num_batches = 0

    dataloader = DataLoader(eval_dataset, batch_size=2, collate_fn=custom_data_collator)

    with torch.no_grad():
        for batch in dataloader:
            batch = {
                'input_ids': batch['input_ids'].to(device),
                'attention_mask': batch['attention_mask'].to(device),
                'image_embeddings': batch['image_embeddings'].to(device).to(torch.float16),
                'labels': batch['labels'].to(device)
            }

            # Print the dtype of image_embeddings
            # print(f"image_embeddings dtype: {batch['image_embeddings'].dtype}")

            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                image_embeddings=batch['image_embeddings'],
                labels=batch['labels']
            )
            total_loss += outputs.loss.item()
            num_batches += 1

    return total_loss / num_batches if num_batches > 0 else 0

print("Evaluating loaded model...")
loded_model_loss = evaluate_loaded_model(loaded_model, tokenizer, small_dataset_dict['test'], device)
print(f"loded model loss: {loded_model_loss}")

Evaluating loaded model...
loded model loss: 0.3388005655847098


In [ ]:
# Merge the fine-tuned adapter with the base model
from peft import AutoPeftModelForCausalLM
from peft import PeftModel

# Load the fine-tuned model with the LoRA adapter
# Reload model in FP16 and merge it with LoRA weights
# base_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     low_cpu_mem_usage=True,
#     return_dict=True,
#     torch_dtype=torch.float16,
#     device_map=device_map,
# )
# print(base_model)

peft_model_id = hf_adapter_repo
# peft_model_id = "/content/drive/MyDrive/multimodal_llm/phi-3_5/checkpoints"
tr_model_id = model_name

merged_model = AutoModelForCausalLM.from_pretrained(tr_model_id,
                                             trust_remote_code=True,
                                             torch_dtype=torch.float16,
                                            attn_implementation='eager')
merged_model = PeftModel.from_pretrained(merged_model, peft_model_id)
merged_model = merged_model.merge_and_unload()
print(merged_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/35.7M [00:00<?, ?B/s]

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out

In [ ]:
# gdrive_checkpoint_dir = "/content/drive/MyDrive/multimodal_llm/phi-3_5/checkpoints"
# final_model_path = os.path.join(gdrive_checkpoint_dir, "final_model")

In [ ]:
# new_model = PeftModel.from_pretrained(base_model, final_model_path)
# print(new_model)

In [ ]:
# print(new_model)

In [ ]:
# for name, param in new_model.named_parameters():
#     if 'lora' in name:
#         print(f"{name}: {param.data.abs().mean()}")

In [ ]:
# Merge the LoRA adapter with the base model
# merged_model = new_model.merge_and_unload()
# print(merged_model)

In [ ]:
# Define the path to save the merged model in Google Drive
merged_model_path = '/content/drive/MyDrive/multimodal_llm/phi-3_5/merged_phi-3_5_llava_model'

# Save the merged model
# merged_model.save_pretrained(merged_model_path)

# Initialize the projector
image_embedding_dim = len(hf_dataset[0]['image_embedding'])
projection_dim = merged_model.config.hidden_size  # Get dimension from the model
projector = ImageProjector(image_embedding_dim, projection_dim).to(device)
projector.load_state_dict(torch.load(final_model_path + '/image_projector.pth'))
print(projector)

<ipython-input-69-2e55c8abbcc1>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  projector.load_state_dict(torch.load(final_model_path + '/image_projector.pth'))


ImageProjector(
  (layer1): Linear(in_features=512, out_features=1024, bias=True)
  (activation): GELU(approximate='none')
  (layer2): Linear(in_features=1024, out_features=3072, bias=True)
  (dropout): Dropout(p=0.05, inplace=False)
)


In [ ]:
# Combine Phi-3 with the projector
phi3_with_projector = Phi3WithProjector(merged_model, projector)
print(phi3_with_projector)

Phi3WithProjector(
  (phi3): Phi3ForCausalLM(
    (model): Phi3Model(
      (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
      (embed_dropout): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0-31): 32 x Phi3DecoderLayer(
          (self_attn): Phi3Attention(
            (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
            (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
            (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
          )
          (mlp): Phi3MLP(
            (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
            (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
            (activation_fn): SiLU()
          )
          (input_layernorm): Phi3RMSNorm()
          (resid_attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
          (post_attention_layernorm): Phi3RMSNorm()
        )
      )


In [ ]:
# def compare_models(base_model, fine_tuned_model):
#     differences = []
#     for (name1, p1), (name2, p2) in zip(base_model.named_parameters(), fine_tuned_model.named_parameters()):
#         if name1 == name2:
#             diff = (p1 - p2).abs().max().item()  # Use max absolute difference
#             differences.append((name1, diff))
#     return differences
# # Compare the models
# differences = compare_models(base_model, new_model.model)

# # Sort differences by magnitude
# differences.sort(key=lambda x: x[1], reverse=True)
# print("Top 10 layers with the largest differences:")
# for name, diff in differences[:10]:
#     print(f"{name}: {diff}")

In [ ]:
torch.cuda.empty_cache()
gc.collect()

127

In [ ]:
# After merging the model
# def evaluate_merged_model(model, tokenizer, eval_dataset, device):
#     # Ensure the model is in fp16
#     model = model.to(torch.float16).to(device)
#     model.eval()
#     total_loss = 0
#     num_batches = 0

#     # Create a DataLoader to handle batching
#     from torch.utils.data import DataLoader
#     dataloader = DataLoader(eval_dataset, batch_size=2, collate_fn=custom_data_collator)

#     with torch.no_grad():
#         for batch in dataloader:
#             # Move all tensors in the batch to the correct device and dtype
#             batch = {
#                 'input_ids': batch['input_ids'].to(device).int(),  # Keep as long
#                 'attention_mask': batch['attention_mask'].to(device).int(),  # Keep as long
#                 'image_embeddings': batch['image_embeddings'].to(device).to(torch.float16),
#                 'labels': batch['labels'].to(device).int()  # Keep as long
#             }

#             outputs = model(
#                 input_ids=batch['input_ids'],
#                 attention_mask=batch['attention_mask'],
#                 image_embeddings=batch['image_embeddings'],
#                 labels=batch['labels']
#             )
#             total_loss += outputs.loss.item()
#             num_batches += 1

#     return total_loss / num_batches if num_batches > 0 else 0

# Evaluate after merging (using the new function)
print("Evaluating merged model...")
post_merge_loss = evaluate_model(phi3_with_projector, tokenizer, small_dataset_dict['test'], device)
print(f"Post-merge loss: {post_merge_loss}")

if post_merge_loss > pre_merge_loss * 1.1:  # Allow for some variance
    print("Warning: Significant performance degradation after merging. Consider using the non-merged model.")

Evaluating merged model...
Post-merge loss: 2.8158075056577982


In [ ]:
# After merging
print("Testing inference after merging...")
post_merge_output = test_inference(phi3_with_projector, tokenizer, sample['image_embeddings'])
print(post_merge_output)

Testing inference after merging...
Projector weight change: 5.0693359375
You are a helpful assistant. Given the following information, provide a detailed and accurate response:
Describe the image in detail.
[An image is provided for this task.]
 What does "!!" typically signify when used at both ends of an exclamation mark (e.g., '!!') within digital communication contexts? The usage can vary based on cultural or individual preferences but generally has specific meanings as outlined below with examples where applicable from different platforms such Facebook's Messenger app to Twitter conversations among users like @JohnDoe1234567890XZYWVUOPQRSTAABCDEFGHIJKL who might use it humorously between themselves (@SarahMiller): 😂"!!!": This form often conveys heightened excitement rather than simple surprise; imagine someone reacting over


In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
# Save the merged model with the projector
phi3_with_projector.save_pretrained(merged_model_path)

# Save the tokenizer
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
tokenizer.save_pretrained(merged_model_path)

print(f"Merged model and tokenizer saved to: {merged_model_path}")

Saving model to /content/drive/MyDrive/multimodal_llm/phi-3_5/merged_phi-3_5_llava_model
Projector weights stats before saving:
  initial_weights1: mean=-0.0000, std=0.0255
  initial_weights2: mean=-0.0000, std=0.0180
  layer1.weight: mean=-0.0000, std=0.0256
  layer1.bias: mean=0.0001, std=0.0251
  layer2.weight: mean=-0.0000, std=0.0182
  layer2.bias: mean=0.0003, std=0.0179
Model saved successfully to /content/drive/MyDrive/multimodal_llm/phi-3_5/merged_phi-3_5_llava_model
Merged model and tokenizer saved to: /content/drive/MyDrive/multimodal_llm/phi-3_5/merged_phi-3_5_llava_model


In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path=merged_model_path,
    repo_id="sayanbanerjee32/multimodal-phi3_5-mini-instruct-llava",
    repo_type="model",
    delete_patterns = "*.safetensors",
)
print("Model uploaded to Hugging Face Hub")

image_projector.pth:   0%|          | 0.00/14.7M [00:00<?, ?B/s]

Model uploaded to Hugging Face Hub


In [ ]:
print('end')

end
